# Perplexity playbook

<p>
Mal Minhas, v0.2<br>
01.01.24
</p>

## 1. Introduction

Perplexity is an intelligent search engine with a chatbot sytle interface that leverages popular AI models to offer comprehensive answers to user inquiries.  It is able to search the internet to provide real-time information via a friendly user experience that includes native mobile apps.  The tool is specifically designed to perform as a research companion providing links to underlying source articles unlike ChatGPT in default mode.  Perplexity comes with an API called [`pplx-api`](https://blog.perplexity.ai/blog/introducing-pplx-api) which was introduced in October 2023.  See [here](https://docs.perplexity.ai/docs/getting-started) for a guide on how to set up an API key and get started with `pplx-api`.  The following examples assume you have set up the API key in an environment variable called `PERPLEXITY_API_KEY`.

Supported models are [as follows](https://docs.perplexity.ai/docs/model-cards):
```
Model	                  Context Length       Model Type
codellama-34b-instruct	      16384          Chat Completion
llama-2-70b-chat               4096          Chat Completion
mistral-7b-instruct [2]	       4096 [1]      Chat Completion
mixtral-8x7b-instruct	       4096 [1]      Chat Completion
pplx-7b-chat	               8192          Chat Completion
pplx-70b-chat	               4096          Chat Completion
pplx-7b-online	               4096          Chat Completion
pplx-70b-online	               4096          Chat Completion

[1] We will be increasing the context length of mistral-7b-instruct to 32k tokens (see roadmap).
[2] This model refers to the v0.2 release of mistral-7b-instruct.
```

The only API function currently supported is on `chat_completions` documented [here](https://docs.perplexity.ai/reference/post_chat_completions).  The next section outlines how to use it.

## 2. Basic Usage

Let's start with a few basics:

In [1]:
import os
import requests

API_KEY = os.environ.get("PERPLEXITY_API_KEY")
DEFAULT_MODEL = "mistral-7b-instruct"
DEFAULT_TOKENS = 1024
CONCISE_DETAIL = "Be precise and concise."
MID_LEVEL_DETAIL = "Explain in a few sentences."
GREAT_DETAIL = "Explain in a lot of detail."
DEFAULT_TEMP = 0.0

The following is a convenience function that wraps the Perplexity API allowing different models to be used and also differing levels of depth of response detail:

In [2]:
def getPerplexityResponse(question, model=DEFAULT_MODEL, instruction=MID_LEVEL_DETAIL, max_tokens=DEFAULT_TOKENS, temperature=DEFAULT_TEMP):
    url = "https://api.perplexity.ai/chat/completions"
    payload = {
        "model": model,
        "stream": False,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [
            {
                "role": "system",
                "content": instruction,
            },
            {
                "role": "user",
                "content": question
            }
        ]
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }
    response = requests.post(url, json=payload, headers=headers).json()
    return response.get("choices")[0].get("message").get("content")

Let's test the function on an example query using the default Mistral 7b model and then Llama2.  Note the differences in response timing:

In [3]:
%%time

getPerplexityResponse("I have a broken washing machine and I live in Southamption in the UK.  What can I do to get it mended?  What trade do I need to call?")

CPU times: user 9.52 ms, sys: 5.15 ms, total: 14.7 ms
Wall time: 1.31 s


'In Southampton, UK, if you have a broken washing machine, you should call a professional appliance repair technician. They have the necessary skills and expertise to diagnose and fix common washing machine issues. You can look up local appliance repair services online or check directories like Yell or Thomson Local. Alternatively, you can contact appliance manufacturers or retailers who may offer repair services for their specific brands. Make sure to ask for quotes and availability before scheduling an appointment'

In [4]:
%%time

getPerplexityResponse("I have a broken washing machine and I live in Southamption in the UK.  What can I do to get it mended?  What trade do I need to call?", model='llama-2-70b-chat')

CPU times: user 6.35 ms, sys: 2.08 ms, total: 8.44 ms
Wall time: 2.22 s


"If you have a broken washing machine in Southampton, UK, you can call a local appliance repair service or a professional appliance engineer to fix the issue. You can search online for appliance repair services in your area, or look up local trade directories such as Yell or Checkatrade to find a reputable repair person. Additionally, you can also contact the manufacturer's customer service number to see if they offer repair services or can recommend a local technician."

Let's see how it handles a query with a more detailed response:

In [5]:
%%time

getPerplexityResponse("I have a broken garden gate which is no longer closing properly.  What trade do I need to call to fix it?", instruction=GREAT_DETAIL)

CPU times: user 6.76 ms, sys: 4.58 ms, total: 11.3 ms
Wall time: 4.4 s


"To fix a broken garden gate, you would typically need to hire the services of a professional tradesperson who specializes in carpentry, gate repair, or fencing. Here's a detailed explanation of the process and the reasons why you might need to call one of these trades:\n\n1. Carpenter: A carpenter is a skilled craftsperson who specializes in the design, construction, and repair of wooden structures. They have a deep understanding of various types of wood, joinery techniques, and carpentry tools. If your garden gate is made of wood and has broken hinges, posts, or other structural components, a carpenter would be an excellent choice to repair or replace the damaged parts. They can also help you with any customization or modifications you might want to make to your gate.\n\n2. Gate Repair Specialist: A gate repair specialist is a tradesperson who focuses specifically on the repair and maintenance of various types of gates, including automatic gates, sliding gates, and swing gates. They 

## 3. Streaming interface

tbd